In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
import os
import warnings

# --- Importar función de normalización ---
# ¡Importante! Esta es la función que te corresponde (Pipeline C)
try:
    from normalization_functions import pipeline_c_normalize
    print("Función 'pipeline_c_normalize' importada correctamente.")
except ImportError:
    print("\n¡Error de Importación!")
    exit()

Función 'pipeline_c_normalize' importada correctamente.


In [3]:
# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
# Ruta de resultados específica para tu pipeline
RESULTS_PATH = os.path.join("..", "results", "fase_1_pipeline_C.csv")

# Asegurarse que la carpeta de resultados exista
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# Ignorar warnings de convergencia de LogisticRegression para una salida limpia
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# --- 2. Definir "Ingredientes" del Experimento ---

# Las 3 representaciones de texto (iguales para todos)
vectorizers = {
    "Binary": CountVectorizer(binary=True),
    "Frequency": CountVectorizer(binary=False),
    "TF-IDF": TfidfVectorizer()
}

# Los 2 modelos simples (iguales para todos)
models = {
    "MultinomialNB": MultinomialNB(),
    # Aumentar max_iter para que LogisticRegression converja con estos datos
    "LogisticRegression": LogisticRegression(max_iter=2000, random_state=0)
}

# --- 3. Cargar y Preparar Datos ---
print(f"Iniciando Experimento: Pipeline C (Lematización)")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo!")
    print(f"No se encontró '{TRAIN_PATH}'.")
    print("Asegúrate de haber corrido '0_split_data.py' primero.")
    exit()

print("Aplicando 'pipeline_c_normalize' a los datos de entrenamiento...")
# Aplicar la normalización C (tu tarea) a toda la columna 'text'
X_train = df_train['text'].apply(pipeline_c_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados. {len(X_train)} documentos listos.")

Iniciando Experimento: Pipeline C (Lematización)
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_c_normalize' a los datos de entrenamiento...
Datos normalizados. 24169 documentos listos.


In [4]:
# --- 4. Bucle de Experimentación (Cross-Validation) ---
print("\nIniciando bucle de validación cruzada (CV=5)...")

experiment_results = []

# Iterar sobre las 3 representaciones
for vec_name, vectorizer in vectorizers.items():
    
    # Iterar sobre los 2 modelos
    for model_name, model in models.items():
        
        # Crear un Pipeline de scikit-learn
        # Esto junta la vectorización y la clasificación en un solo objeto
        # Es la forma *correcta* de usar cross_validate,
        # ya que asegura que la vectorización se aprenda en cada fold.
        text_pipeline = Pipeline([
            ('vectorizer', vectorizer), # Paso 1: Convertir texto a números
            ('classifier', model)      # Paso 2: Clasificar
        ])
        
        print(f"-> Ejecutando: (Pipeline C) + ({vec_name}) + ({model_name})")
        
        # Ejecutar la validación cruzada de 5 folds 
        # y calcular el f1_macro 
        cv_scores = cross_validate(
            estimator=text_pipeline,
            X=X_train, # Los textos ya normalizados con Pipeline C
            y=y_train,
            cv=5,               
            scoring='f1_macro', 
            n_jobs=-1           # Usar todos los núcleos de CPU
        )
        
        # Calcular el promedio y desviación de los 5 scores
        avg_f1 = cv_scores['test_score'].mean()
        std_f1 = cv_scores['test_score'].std()
        
        print(f"   -> F1-Macro Promedio: {avg_f1:.4f} (std: {std_f1:.4f})")
        
        # Guardar el resultado
        experiment_results.append({
            "pipeline": "C (B + Lematización)", 
            "vectorizer": vec_name,
            "model": model_name,
            "avg_f1_macro": avg_f1,
            "std_f1_macro": std_f1
        })

# --- 5. Guardar y Reportar Resultados ---
results_df = pd.DataFrame(experiment_results)

# Guardar tu trabajo en un CSV en la carpeta de resultados
try:
    results_df.to_csv(RESULTS_PATH, index=False)
except Exception as e:
    print(f"\n¡Error al guardar resultados! {e}")
    
print("\n--- Resultados Finales (Pipeline C) ---")
# Imprimir los resultados ordenados por el mejor F1
print(results_df.sort_values(by="avg_f1_macro", ascending=False).to_string())
print(f"\n¡Experimento completo! Resultados guardados en '{RESULTS_PATH}'")


Iniciando bucle de validación cruzada (CV=5)...
-> Ejecutando: (Pipeline C) + (Binary) + (MultinomialNB)
   -> F1-Macro Promedio: 0.2796 (std: 0.0062)
-> Ejecutando: (Pipeline C) + (Binary) + (LogisticRegression)
   -> F1-Macro Promedio: 0.4508 (std: 0.0170)
-> Ejecutando: (Pipeline C) + (Frequency) + (MultinomialNB)
   -> F1-Macro Promedio: 0.3095 (std: 0.0064)
-> Ejecutando: (Pipeline C) + (Frequency) + (LogisticRegression)
   -> F1-Macro Promedio: 0.4566 (std: 0.0146)
-> Ejecutando: (Pipeline C) + (TF-IDF) + (MultinomialNB)
   -> F1-Macro Promedio: 0.1638 (std: 0.0002)
-> Ejecutando: (Pipeline C) + (TF-IDF) + (LogisticRegression)
   -> F1-Macro Promedio: 0.3910 (std: 0.0097)

--- Resultados Finales (Pipeline C) ---
               pipeline vectorizer               model  avg_f1_macro  std_f1_macro
3  C (B + Lematización)  Frequency  LogisticRegression      0.456558      0.014604
1  C (B + Lematización)     Binary  LogisticRegression      0.450816      0.016953
5  C (B + Lematización